<a href="https://colab.research.google.com/github/hoky1227/Transformer_based-recommendation/blob/main/Transformer_Instacart_with_ECF_Algorithm_1230_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Instacart/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Module 가져오기

In [2]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tqdm import tqdm
import math
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

import warnings 

warnings.filterwarnings('ignore')

### Data Road

In [3]:
TFoutputs = pd.read_csv(path + "Instacart_Transformer_test_outputs_20.csv")
# TFoutputs.rename(columns={'acc':'predict_item'}, inplace=True)
TFoutputs.pop('Unnamed: 0')
TFoutputs

,train,label,split_train_token,split_label_token,predict_item
0,Health Super Lysine+ Immune System Liquid Extr...,Wellness Formula,<esp> Health Super Lysine+ Immune System Liqui...,<esp> Wellness Formula <esp>,<esp> Trilogy Kombucha Drink <esp>
1,Tiny Fruits Blueberry Apple&&Organic Freeze Dr...,Naturals Chicken Nuggets,<esp> Tiny Fruits Blueberry Apple <esp> Organi...,<esp> Naturals Chicken Nuggets <esp>,<esp> Organic Granny Smith Apple <esp>
2,Organic Pure Lime Juice&&Pure Lemon Juice&&Uns...,Cashew Milk Salted Caramel Cluster Non-Dairy F...,<esp> Organic Pure Lime Juice <esp> Pure Lemon...,<esp> Cashew Milk Salted Caramel Cluster Non-D...,<esp> Organic Baby Spinach <esp>
3,Organic Lemon&&Bean & Rice Chips Nacho&&Gluten...,Compostable Basket Coffee Filters,<esp> Organic Lemon <esp> Bean & Rice Chips Na...,<esp> Compostable Basket Coffee Filters <esp>,<esp> Crackers Cheddar Bunnies Snack Packs <esp>
4,Roasted Turkey&&Natural Free & Clear Dishwashe...,Banana,<esp> Roasted Turkey <esp> Natural Free & Clea...,<esp> Banana <esp>,<esp> Organic Whole Wheat Fusilli <esp>
...,...,...,...,...,...
27430,Cheddar Snack Crackers Cheddar Bunnies&&Olive ...,Banana,<esp> Cheddar Snack Crackers Cheddar Bunnies <...,<esp> Banana <esp>,<esp> Organic Whole Milk <esp>
27431,Frozen Organic Wild Blueberries&&Organic Whole...,Organic Whole Milk,<esp> Frozen Organic Wild Blueberries <esp> Or...,<esp> Organic Whole Milk <esp>,<esp> Organic Baby Spinach <esp>
27432,Organic Raspberries&&Frozen Organic Wild Blueb...,Organic Whole Strawberries,<esp> Organic Raspberries <esp> Frozen Organic...,<esp> Organic Whole Strawberries <esp>,<esp> Organic Baby Spinach <esp>
27433,"Turkey Bacon&&Egg, Basil Pesto, & Cheese Scram...",Deli Fresh Slow Roasted Cured Roast Beef,"<esp> Turkey Bacon <esp> Egg, Basil Pesto, & C...",<esp> Deli Fresh Slow Roasted Cured Roast Beef...,<esp> Pesto Tortellini Bowls <esp>


In [4]:
Detail = pd.read_csv(path + "InstacartDetail.csv")
Detail.pop('Unnamed: 0')
# print(len(detail))
# detail = detail['0'].to_list()
detail = Detail['product_name'].str.strip().to_list()
len(detail)

49688

In [5]:
pred = TFoutputs['predict_item']
pred = pred.str.replace('<esp>', '')
pred = pred.str.strip()
pred.head()

0                  Trilogy Kombucha Drink
1              Organic Granny Smith Apple
2                    Organic Baby Spinach
3    Crackers Cheddar Bunnies Snack Packs
4             Organic Whole Wheat Fusilli
Name: predict_item, dtype: object

In [6]:
# checker = pd.Series([i in list(TFoutputs['split_label_token']) for i in TFoutputs['predict_item']])
checker = pd.Series([i in detail for i in pred])
# filter = checker.replace(False, np.NaN)
print(checker.value_counts())
filter = checker[checker == False].index
len(filter)

True     27117
False      318
dtype: int64


318

In [7]:
TFoutputs['filter'] = checker
# falses = TFoutputs.isna().sum()
# TFoutputs.dropna(inplace=True)
# TFoutputs.reset_index(drop=True, inplace=True)
# TFoutputs

In [8]:
TFoutputs

,train,label,split_train_token,split_label_token,predict_item,filter
0,Health Super Lysine+ Immune System Liquid Extr...,Wellness Formula,<esp> Health Super Lysine+ Immune System Liqui...,<esp> Wellness Formula <esp>,<esp> Trilogy Kombucha Drink <esp>,True
1,Tiny Fruits Blueberry Apple&&Organic Freeze Dr...,Naturals Chicken Nuggets,<esp> Tiny Fruits Blueberry Apple <esp> Organi...,<esp> Naturals Chicken Nuggets <esp>,<esp> Organic Granny Smith Apple <esp>,True
2,Organic Pure Lime Juice&&Pure Lemon Juice&&Uns...,Cashew Milk Salted Caramel Cluster Non-Dairy F...,<esp> Organic Pure Lime Juice <esp> Pure Lemon...,<esp> Cashew Milk Salted Caramel Cluster Non-D...,<esp> Organic Baby Spinach <esp>,True
3,Organic Lemon&&Bean & Rice Chips Nacho&&Gluten...,Compostable Basket Coffee Filters,<esp> Organic Lemon <esp> Bean & Rice Chips Na...,<esp> Compostable Basket Coffee Filters <esp>,<esp> Crackers Cheddar Bunnies Snack Packs <esp>,True
4,Roasted Turkey&&Natural Free & Clear Dishwashe...,Banana,<esp> Roasted Turkey <esp> Natural Free & Clea...,<esp> Banana <esp>,<esp> Organic Whole Wheat Fusilli <esp>,True
...,...,...,...,...,...,...
27430,Cheddar Snack Crackers Cheddar Bunnies&&Olive ...,Banana,<esp> Cheddar Snack Crackers Cheddar Bunnies <...,<esp> Banana <esp>,<esp> Organic Whole Milk <esp>,True
27431,Frozen Organic Wild Blueberries&&Organic Whole...,Organic Whole Milk,<esp> Frozen Organic Wild Blueberries <esp> Or...,<esp> Organic Whole Milk <esp>,<esp> Organic Baby Spinach <esp>,True
27432,Organic Raspberries&&Frozen Organic Wild Blueb...,Organic Whole Strawberries,<esp> Organic Raspberries <esp> Frozen Organic...,<esp> Organic Whole Strawberries <esp>,<esp> Organic Baby Spinach <esp>,True
27433,"Turkey Bacon&&Egg, Basil Pesto, & Cheese Scram...",Deli Fresh Slow Roasted Cured Roast Beef,"<esp> Turkey Bacon <esp> Egg, Basil Pesto, & C...",<esp> Deli Fresh Slow Roasted Cured Roast Beef...,<esp> Pesto Tortellini Bowls <esp>,True


In [9]:
# falses = falses['filter']
# falses

falses = pd.Series([TFoutputs['predict_item'][i] for i in filter])
falses = falses.str.replace('<esp>', '')
falses = falses.str.strip()
print(len(falses))
falses.head()

318


0                Original Strawberry Banana Juice
1       Finely Shredded Mexican Four Cheese Blend
2                           Organic Turmeric Root
3                Natural Peanut Butter Flavor Bar
4    Butcher Thick Cut Shredded Mozzarella Cheese
dtype: object

In [10]:
filter[:5]

Int64Index([42, 100, 269, 274, 281], dtype='int64')

In [11]:
df = pd.read_pickle(path + "InstacartCustomers.pkl")
# df.pop('Unnamed: 0')
df

,user,products
0,1,Organic Fuji Apples&&Bag of Organic Bananas&&H...
1,2,Organic Roasted Sliced Chicken Breast&&Articho...
2,3,"Crackers, Crispy, Cheddar&&Organic Baby Spinac..."
3,4,Original Orange Juice&&Extra-Dry Cava&&Organic...
4,5,"Semi-Soft Cheese, Ripened&&Red Raspberries&&Or..."
...,...,...
206204,206205,Hass Avocado&&Organic Whole String Cheese&&Ora...
206205,206206,Large Alfresco Eggs&&Gluten Free Apple Cinnamo...
206206,206207,Sparkling Water Grapefruit&&Organic Raspberrie...
206207,206208,Organic Whipping Cream&&Organic Lemon&&Peanut ...


In [12]:
user2item = df.copy()

In [13]:
# user2item.columns = ['유저 식별키', '유저 구매 상품 식별키 묶음', '라벨']
user2item.columns = ['유저 식별키', '유저 구매 상품 식별키 묶음']
user2item

,유저 식별키,유저 구매 상품 식별키 묶음
0,1,Organic Fuji Apples&&Bag of Organic Bananas&&H...
1,2,Organic Roasted Sliced Chicken Breast&&Articho...
2,3,"Crackers, Crispy, Cheddar&&Organic Baby Spinac..."
3,4,Original Orange Juice&&Extra-Dry Cava&&Organic...
4,5,"Semi-Soft Cheese, Ripened&&Red Raspberries&&Or..."
...,...,...
206204,206205,Hass Avocado&&Organic Whole String Cheese&&Ora...
206205,206206,Large Alfresco Eggs&&Gluten Free Apple Cinnamo...
206206,206207,Sparkling Water Grapefruit&&Organic Raspberrie...
206207,206208,Organic Whipping Cream&&Organic Lemon&&Peanut ...


# Product2Vector 생성

### Jaccard 유사도

In [14]:
# pd.Series([i in detail for i in falses]).value_counts()

In [15]:
def JaccardSimilarity(inp1, inp2):
    list_inp1 = inp1.split()
    list_inp2 = inp2.split()
    mom = set(list_inp1).union(set(list_inp2))
    son = set(list_inp1).intersection(set(list_inp2))
    # print(mom)
    # print(son)
    return len(son)/len(mom)

In [16]:
# detail = TFoutputs['label'].unique().tolist()
# print(detail[-1])
# len(detail)

In [17]:
pred[filter[0]]

'Original Strawberry Banana Juice'

In [18]:
asdf = []

for v in tqdm(falses):
    max = 0
    for d in detail:
        score = JaccardSimilarity(v, d)
        if max < score:
            max = score
            temp = d
    if max == 0: asdf.append(v)
    id = pred[pred == v].index
    for i in id:
        pred[i] = temp

100%|██████████| 318/318 [01:02<00:00,  5.06it/s]


In [19]:
len(asdf)

0

In [20]:
falses[0]

'Original Strawberry Banana Juice'

In [21]:
pred[filter[0]]

'Strawberry Banana Juice'

In [22]:
pd.Series([i in detail for i in pred]).value_counts()

True    27435
dtype: int64

In [23]:
changes = pd.DataFrame(columns=['Original', 'Changed'])
changes['Original'] = falses
changes['Changed'] = pred[filter].to_list()
changes

,Original,Changed
0,Original Strawberry Banana Juice,Strawberry Banana Juice
1,Finely Shredded Mexican Four Cheese Blend,Cheese Finely Shredded Mexican Four Cheese Blend
2,Organic Turmeric Root,Organic Ground Turmeric Root
3,Natural Peanut Butter Flavor Bar,Peanut Butter Natural Protein Bar
4,Butcher Thick Cut Shredded Mozzarella Cheese,Mozzarella Fine Cut Shredded Cheese
...,...,...
313,White 2% Reduced Fat Milk,2% Reduced Fat Milk
314,Organic Coconut Bliss Salted Caramel Granola Bars,Organic Coconut Bliss Salted Caramel in Chocolate
315,Original Thin Crust Pizza,Original Thin Crust Pepperoni Pizza
316,Graduates Lil' Entrees with Carrots Mashed Pot...,Graduates Lil' Entrees with Carrots Mashed Pot...


In [24]:
changes.to_csv(path + 'Instacart_changed_products_1.csv', mode='w', encoding='UTF-8')

### Word2vec Model 학습

In [26]:
model = Word2Vec([item_list.split('&&') for item_list in user2item['유저 구매 상품 식별키 묶음']], size=100, window=5, min_count=1, negative=15, iter=10, sample=0.00001, sg=1)

In [28]:
model.save(path + 'Instacart_word2vec.model')

In [ ]:
# # 유저 성향 벡터 구하기 
# user_propensity_vector = []
# for item_list in tqdm_notebook(user2item['유저 구매 상품 식별키 묶음']):
#     item_list = item_list.split('&&')
#     bbb = np.array(np.ones(100))
#     for aaa in item_list:
#         bbb *= model[aaa]
#         bbb = bbb/norm(bbb)
#     user_propensity_vector.append(bbb)

In [30]:
# 유사 product 찾기
# def find_item(user, n=10):
#     item_code = user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('&&')[-1]
#     return list(dict(model.wv.most_similar(item_code, topn=n)))

def find_item(item, n=10):
    return list(dict(model.wv.most_similar(item, topn=n)))

# 유저 유사도 계산 및 유사 유저 찾기
def user_sim(A, B): 
    A_vec = user_propensity_vector[user2item[user2item['유저 식별키']==A].index[0]]
    B_vec = user_propensity_vector[user2item[user2item['유저 식별키']==B].index[0]]
    return euclidean(A_vec.reshape(-1, 1), B_vec.reshape(-1, 1))

def find_user(user):
    user_index = user2item[user2item['유저 식별키']==user].index[0]
    min_user_similar = min(user_matrix_f[user_index].drop(user_index))
    return user2item.iloc[{j:i for i, j in enumerate(user_matrix_f[user_index])}[min_user_similar]]['유저 식별키']

In [31]:
# # 유저 유사도 매트릭스
# user_matrix_f = pd.DataFrame()
# for i in tqdm_notebook(range(len(user_propensity_vector))):
#     tmp = []
#     for j in range(len(user_propensity_vector)):
#         tmp.append(euclidean(user_propensity_vector[i], user_propensity_vector[j]))
#     user_matrix_f = pd.concat([user_matrix_f, pd.DataFrame(tmp)], axis = 1)
# user_matrix_f.columns = user_matrix_f.index
# user_matrix_f

In [32]:
find_item(pred[0])

['Synergy Organic Kombucha Gingerberry',
 'Organic Raw Kombucha Gingerade',
 'Organic Raw Multigreen Kobmbucha',
 'Kombucha, Organic Raw, Citrus',
 'Organic & Raw Strawberry Serenity Kombucha',
 'Synergy Organic & Raw Cosmic Cranberry',
 'Passionberry Bliss Kombucha Drink',
 'Enlightened Organic Raw Kombucha',
 'Guava Goddess Organic Kambucha',
 'Hibiscus Organic Raw Kombucha']

## 구매전 : Top-10 시나리오

In [39]:
def recommend(n=10, acc=False, pred=pred, TFoutputs=TFoutputs):
    item_list = []
    for v in tqdm_notebook(pred.to_list()):
        if n > 1:
            pp = find_item(v, n-1)
            pp.append(v)
            item_list.append(pp)
        else: item_list.append(v)
    TFoutputs['추천 list'] = item_list
    # user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]
    TFoutputs['정답 유무'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['추천 list']), total=(len(TFoutputs)))]
    # 추천 알고리듬 구매전 Top-10 성능평가
    
    if acc:
        return TFoutputs[TFoutputs['정답 유무']==True].shape[0]/(TFoutputs.shape[0])

    return TFoutputs

In [36]:
recommend(n=3, acc=True)

0.02711864406779661

In [37]:
recommend(n=5, acc=True)

0.031346819755786404

In [40]:
recommend(n=10, acc=True)

0.04239110625113906

In [41]:
recommend(n=20, acc=True)

0.052925095680699835

In [42]:
recommend()

,train,label,split_train_token,split_label_token,predict_item,filter,추천 list,정답 유무
0,Health Super Lysine+ Immune System Liquid Extr...,Wellness Formula,<esp> Health Super Lysine+ Immune System Liqui...,<esp> Wellness Formula <esp>,<esp> Trilogy Kombucha Drink <esp>,True,"[Synergy Organic Kombucha Gingerberry, Organic...",False
1,Tiny Fruits Blueberry Apple&&Organic Freeze Dr...,Naturals Chicken Nuggets,<esp> Tiny Fruits Blueberry Apple <esp> Organi...,<esp> Naturals Chicken Nuggets <esp>,<esp> Organic Granny Smith Apple <esp>,True,"[Organic Navel Orange, Organic D'Anjou Pears, ...",False
2,Organic Pure Lime Juice&&Pure Lemon Juice&&Uns...,Cashew Milk Salted Caramel Cluster Non-Dairy F...,<esp> Organic Pure Lime Juice <esp> Pure Lemon...,<esp> Cashew Milk Salted Caramel Cluster Non-D...,<esp> Organic Baby Spinach <esp>,True,"[Organic Unsweetened Almond Milk, Organic Froz...",False
3,Organic Lemon&&Bean & Rice Chips Nacho&&Gluten...,Compostable Basket Coffee Filters,<esp> Organic Lemon <esp> Bean & Rice Chips Na...,<esp> Compostable Basket Coffee Filters <esp>,<esp> Crackers Cheddar Bunnies Snack Packs <esp>,True,"[Organic AppleApple, Apple Cinnamon GoGo Squee...",False
4,Roasted Turkey&&Natural Free & Clear Dishwashe...,Banana,<esp> Roasted Turkey <esp> Natural Free & Clea...,<esp> Banana <esp>,<esp> Organic Whole Wheat Fusilli <esp>,True,"[Organic Whole Wheat Penne Rigate, Organic Who...",False
...,...,...,...,...,...,...,...,...
27430,Cheddar Snack Crackers Cheddar Bunnies&&Olive ...,Banana,<esp> Cheddar Snack Crackers Cheddar Bunnies <...,<esp> Banana <esp>,<esp> Organic Whole Milk <esp>,True,"[Organic Frozen Peas, Organic Strawberries, Or...",False
27431,Frozen Organic Wild Blueberries&&Organic Whole...,Organic Whole Milk,<esp> Frozen Organic Wild Blueberries <esp> Or...,<esp> Organic Whole Milk <esp>,<esp> Organic Baby Spinach <esp>,True,"[Organic Unsweetened Almond Milk, Organic Froz...",False
27432,Organic Raspberries&&Frozen Organic Wild Blueb...,Organic Whole Strawberries,<esp> Organic Raspberries <esp> Frozen Organic...,<esp> Organic Whole Strawberries <esp>,<esp> Organic Baby Spinach <esp>,True,"[Organic Unsweetened Almond Milk, Organic Froz...",False
27433,"Turkey Bacon&&Egg, Basil Pesto, & Cheese Scram...",Deli Fresh Slow Roasted Cured Roast Beef,"<esp> Turkey Bacon <esp> Egg, Basil Pesto, & C...",<esp> Deli Fresh Slow Roasted Cured Roast Beef...,<esp> Pesto Tortellini Bowls <esp>,True,"[Broccoli & Cheddar Bake Meal Bowl, Cheese Rav...",False


In [ ]:
# item_list = []
# for v in pred.to_list():
#     pp = find_item(v)
#     item_list.append(pp)

# item_list[:2]

In [ ]:
# def recommendation(user):
#     n = 10
#     while True:
#         recom_item = set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('&&')) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('&&'))
#         n += 10
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]           #Top10
#         if n >= 1000:
#             return list(recom_item)

In [ ]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(user2item['유저 식별키'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':user2item['유저 식별키'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','&&') 
#                                    for i in rec_list]})
# user2rec

In [ ]:
# TFoutputs['추천 list'] = item_list

In [ ]:
# # user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]
# TFoutputs['정답 유무'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['추천 list']), total=(len(TFoutputs)))]

In [ ]:
# TFoutputs

In [ ]:
# # 추천 알고리듬 구매전 Top-10 성능평가

# TFoutputs[TFoutputs['정답 유무']==True].shape[0]/(TFoutputs.shape[0] + falses)

# 구매후(교-차)  : Top-10 시나리오

In [ ]:
# def recommendation(user):
#     n = 10
#     while True:
#         recom_item = set(set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('@')) & set(find_item(user, n))) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('@'))
#         n += 10   
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]            #Top10
#         if n >= 1000:
#             return list(recom_item)

In [ ]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(df_f2['고객번호'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':df_f2['고객번호'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','@') 
#                                     for i in rec_list]})
# user2rec

In [ ]:
# user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]

In [ ]:
# # 추천 알고리듬 구매후(교-차) Top-10 성능평가


# user2item[user2item['정답 유무']==True].shape[0]/user2item.shape[0]*100 #Top10

# 구매후(합-차) : Top-10 시나리오

In [ ]:
# def recommendation(user):
#     n = 20
#     while True:
#         recom_item = set(set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('@')) | set(find_item(user, n))) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('@'))
#         n += 10   
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]              #Top10
#         if n >= 1000:
#             return list(recom_item)

In [ ]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(df_f2['고객번호'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':df_f2['고객번호'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','@') 
#                                     for i in rec_list]})
# user2rec

In [ ]:
# user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]

In [ ]:
# # 추천 알고리듬 구매후(합-차) Top-10 성능평가

# user2item[user2item['정답 유무']==True].shape[0]/user2item.shape[0]*100 #Top10